# Global and local scopes


In [11]:
#local scope
a=0
b=1
def func():
    a=100
    print(a)
    print(b)
func()
print(a)

100
1
0


In [9]:
#Global scope
a=0
def func():
    global a
    a=100
    print(a)
func()
print(a)

100
100


## Nonlocal Scopes

In [13]:
def outer_func():
    a=10
    def inner_func():
        print(a)
    inner_func()
outer_func()

10


In [14]:
def outer_func():
    a=10
    def inner1():
        def inner2():
            nonlocal a
            a=100
        inner2()
    inner1()
    print(a)
outer_func()

100


In [15]:
def outer_func():
    a='outer'
    def inner1():
        nonlocal a
        a='inner1'
        def inner2():
            nonlocal a
            a='inner2'
        inner2()
    inner1()
    print(a)
outer_func()

inner2


## Closures

In [32]:
def counter():
    #closure starts
    count=0
    def inner():
        nonlocal count
        count+=1
        return count
        #closure ends
    return inner
f1 = counter()
f2 = counter()
print("f1:{0}".format(f1()))
print("f1:{0}".format(f1()))
print("f2:{0}".format(f2()))
print("f1:{0}".format(f1()))
print("f2:{0}".format(f2()))
print("f2:{0}".format(f2()))




f1:1
f1:2
f2:1
f1:3
f2:2
f2:3


In [3]:
#share free variable between 2 different closures
def func_outer():
    count=0
    def inc1():
        nonlocal count 
        count+=1
        return count
    def inc2():
        nonlocal count
        count+=1
        return count
    return inc1, inc2
f1, f2 = func_outer()
print(f1())
print(f2())
print(f1())
print(f2())

1
2
3
4


In [38]:
#shared extended scopes Caution!
adders=[]
for n in range(1,4):
    #n is the free variable that creates the closure with lambda
    adders.append(lambda x: x+n)
print(adders[0](10))
print(adders[1](10))
print(adders[2](10))

13
13
13


In [48]:
#nested closures
def incrementer(n):
    #inner + n is a closure
    def inner(start):
        current=start
        #inner + current is a closure
        def inc():
            nonlocal current
            current+=n
            return current
        return inc
    return inner

fn = incrementer(2)
inc_2 = fn(100)
print(fn.__code__.co_freevars)
print(inc_2.__code__.co_freevars)
print(fn(100))
print(inc_2())


('n',)
('current', 'n')
<function incrementer.<locals>.inner.<locals>.inc at 0x0000027AA7242798>
102


## closure applications

In [56]:
def averager():
    numbers = []
    def add(number):
        numbers.append(number)
        total = sum(numbers)
        count = len(numbers)
        return total/count
    return add
a = averager()
b = averager()
print(a(10))
print(b(20))
print(a(20))
print(b(30))
print(a.__closure__)
print(b.__closure__)

10.0
20.0
15.0
25.0
(<cell at 0x0000027AA8140D38: list object at 0x0000027AA84B5788>,)
(<cell at 0x0000027AA8140CD8: list object at 0x0000027AA7FEB048>,)


## emulate closure with a class

In [62]:
class Averager:
    def __init__(self):
        self.total=0
        self.count=0
    def add(self, number):
        self.total+=number
        self.count+=1
        return self.total/self.count

def averager():
    total = 0
    count = 0
    def add(number):
        nonlocal total
        nonlocal count
        total += number
        count += 1
        return total/count
    return add

a = Averager()
b = averager()
print(a.add(10))
print(a.add(20))
print(b(10))
print(b(20))


10.0
15.0
10.0
15.0


In [65]:
from time import perf_counter

class Timer:
    def __init__(self):
        self.start = perf_counter()
    
    def __call__(self):
        return perf_counter() -self.start

t1 = Timer()


In [66]:
t1()

3.5869563999985985

In [67]:
t1()

13.842820300000312

In [70]:
def timer():
        start = perf_counter()
        def poll():
            return perf_counter() - start
        return poll
t2 = timer()

In [71]:
t2()

2.3391418000028352

In [72]:
t2()

7.867270100003225

In [76]:
def counter(initial_value=0):
    def my_closure(increment=1):
        nonlocal initial_value
        initial_value+=increment
        return initial_value
    return my_closure

c1 = counter()
c2 = counter(100)
print(c1())
print(c1())
print(c2())
print(c2())

1
2
101
102


## Decorators

In [5]:
# counter is the decorator fucntion, it adds extra functionality to my_func
# functools.wraps allows to keep the original meta data of a function after decoration is applied
from functools import wraps 
def counter(fn):
    count=0
    @wraps(fn)
    def inner(*args,**kwargs):
        nonlocal count
        count+=1
        print('Function {0} was called {1} times'.format(fn.__name__, count))
        return fn(*args,**kwargs)
    return inner

def my_func(a,b=0):
    return a+b

@counter
def decorated(a,b=0):
    """ this is the documentation 
    for decorated function """
    return a+b

my_func = counter(my_func)
result = my_func(1,2)
result2 = decorated(1,2)
result3 = decorated(1,5)
print('_____________')
print(result)
print('_____________')
print(result2)
print('_____________')
print(help(decorated))

Function my_func was called 1 times
Function decorated was called 1 times
Function decorated was called 2 times
_____________
3
_____________
3
_____________
Help on function decorated in module __main__:

decorated(a, b=0)
    this is the documentation 
    for decorated function

None


## Stacked Decorators

In [11]:
def timed(fn):
    from functools import wraps
    from time import perf_counter

    @wraps(fn)
    def inner(*args,**kwargs):
        start = perf_counter()
        result = fn(*args,**kwargs)
        end = perf_counter()
        print('{0}: ran for {1:.6f}s'.format(fn.__name__, end-start))
        return result
    return inner

def logged(fn):
    from functools import wraps
    from datetime import datetime, timezone

    @wraps(fn)
    def inner(*args,**kwargs):
        run_dt = datetime.now(timezone.utc)
        result = fn(*args,**kwargs)
        print('{0}: called {1}'.format(run_dt, fn.__name__))
        return result
    return inner

@logged
@timed
def func1():
    pass

@timed
@logged
def func2():
    pass

print(func1())
print(func2())

func1: ran for 0.000001s
2021-04-11 22:14:02.616830+00:00: called func1
None
2021-04-11 22:14:02.616830+00:00: called func2
func2: ran for 0.000034s
None


# Memoization

In [15]:
#recursive but inefficient way to calculate fibo seq.
def fib(n):
    print('calculating fib({0})'.format(n))
    return 1 if n<3 else fib(n-1)+fib(n-2)
fib(10)

calculating fib(10)
calculating fib(9)
calculating fib(8)
calculating fib(7)
calculating fib(6)
calculating fib(5)
calculating fib(4)
calculating fib(3)
calculating fib(2)
calculating fib(1)
calculating fib(2)
calculating fib(3)
calculating fib(2)
calculating fib(1)
calculating fib(4)
calculating fib(3)
calculating fib(2)
calculating fib(1)
calculating fib(2)
calculating fib(5)
calculating fib(4)
calculating fib(3)
calculating fib(2)
calculating fib(1)
calculating fib(2)
calculating fib(3)
calculating fib(2)
calculating fib(1)
calculating fib(6)
calculating fib(5)
calculating fib(4)
calculating fib(3)
calculating fib(2)
calculating fib(1)
calculating fib(2)
calculating fib(3)
calculating fib(2)
calculating fib(1)
calculating fib(4)
calculating fib(3)
calculating fib(2)
calculating fib(1)
calculating fib(2)
calculating fib(7)
calculating fib(6)
calculating fib(5)
calculating fib(4)
calculating fib(3)
calculating fib(2)
calculating fib(1)
calculating fib(2)
calculating fib(3)
calculating

55

In [26]:
# class and closure way to calculate fibo seq
class Fib:
    def __init__(self):
        self.cache = {1: 1,2: 1} #first 2 fib numbers
    
    def fib(self, n):
        if n not in self.cache:
            print('calculating fib({0})'.format(n))
            self.cache[n] = self.fib(n-1) + self.fib(n-2)
        return self.cache[n]

def closure_fibo():
    cache = {1:1,2:1}
    def calc_fib(n):
        if n not in cache:
            print('calculating fib({0})'.format(n))
            cache[n]=calc_fib(n-1)+calc_fib(n-2)
        return cache[n]
    return calc_fib

f = Fib()
f2 =closure_fibo()
print(f.fib(10))
print('--------------')
print(f2(10))

calculating fib(10)
calculating fib(9)
calculating fib(8)
calculating fib(7)
calculating fib(6)
calculating fib(5)
calculating fib(4)
calculating fib(3)
55
--------------
calculating fib(10)
calculating fib(9)
calculating fib(8)
calculating fib(7)
calculating fib(6)
calculating fib(5)
calculating fib(4)
calculating fib(3)
55


In [15]:
# memoization way
def memoize(fn):
    cache = dict()
    def inner(n):
        if n not in cache:
            cache[n]=fn(n)
        return cache[n]
    return inner

@memoize
def fibonacci(n):
    print('calculating fib({0})'.format(n))
    return 1 if n < 3 else fibonacci(n-1) + fibonacci(n-2)

@memoize
def factorial(n):
    print('calculating {0}!'.format(n))
    return 1 if n < 2 else n*factorial(n-1)

print(fibonacci(10))
print(fibonacci(10))
print(fibonacci(11))
print(factorial(4))
print(factorial(6))

calculating fib(10)
calculating fib(9)
calculating fib(8)
calculating fib(7)
calculating fib(6)
calculating fib(5)
calculating fib(4)
calculating fib(3)
calculating fib(2)
calculating fib(1)
55
55
calculating fib(11)
89
calculating 4!
calculating 3!
calculating 2!
calculating 1!
24
calculating 6!
calculating 5!
720


In [25]:
# memoization built-in function
from functools import lru_cache

@lru_cache()
def fibonacci(n):
    print('calculating fib({0})'.format(n))
    return 1 if n < 3 else fibonacci(n-1) + fibonacci(n-2)

@lru_cache(maxsize=3)
def factorial(n):
    print('calculating {0}!'.format(n))
    return 1 if n < 2 else n*factorial(n-1)

print(factorial(4))
print(factorial(7))
print(factorial(4))
help(lru_cache)

calculating 4!
calculating 3!
calculating 2!
calculating 1!
24
calculating 7!
calculating 6!
calculating 5!
5040
calculating 4!
calculating 3!
calculating 2!
calculating 1!
24
Help on function lru_cache in module functools:

lru_cache(maxsize=128, typed=False)
    Least-recently-used cache decorator.
    
    If *maxsize* is set to None, the LRU features are disabled and the cache
    can grow without bound.
    
    If *typed* is True, arguments of different types will be cached separately.
    For example, f(3.0) and f(3) will be treated as distinct calls with
    distinct results.
    
    Arguments to the cached function must be hashable.
    
    View the cache statistics named tuple (hits, misses, maxsize, currsize)
    with f.cache_info().  Clear the cache and statistics with f.cache_clear().
    Access the underlying function with f.__wrapped__.
    
    See:  http://en.wikipedia.org/wiki/Cache_replacement_policies#Least_recently_used_(LRU)



## Decorator with parameters

In [28]:
def my_dec(a,b):
    def dec(fn):
        def inner(*args,**kwargs):
            print("decorated function parameters: a={0}, b={1}".format(a,b))
            return fn(*args,**kwargs)
        return inner
    return dec
@my_dec(10,20)
def func(s):
    print('Hello {0}'.format(s))

print(func('john'))

decorated function parameters: a=10, b=20
Hello john
None


## Decorator class 

In [34]:
class MyClass:
    def __init__(self,a,b):
        self.a = a
        self.b = b
    def __call__(self,fn):
        def inner(*args,**kwargs):
            print("decorated function parm: a={0}, b={1}".format(self.a,self.b))
            return fn(*args,**kwargs)
        return inner

@MyClass(10,20)
def my_func(s):
    print('Hello {0}'.format(s))
my_func('john')

decorated function parm: a=10, b=20
Hello john


## decorating classes

In [42]:
#adding attributes and methods dynamically to a Class
from fractions import Fraction
f = Fraction(2,3)
print(f.numerator)
print(f.denominator)
Fraction.newproperty = 100
print(f.newproperty)
Fraction.newmethod = lambda self, message: 'hijo de re {0} putas!!'.format(message)
print(f.newmethod('1000'))


2
3
100
hijo de re 1000 putas!!


In [47]:
# using decorator
from datetime import datetime, timezone

def info(self):
    results = []
    results.append('time: {0}'.format(datetime.now(timezone.utc)))
    results.append('class: {0}'.format(self.__class__.__name__))
    results.append('id: {0}'.format(hex(id(self))))
    for k,v in vars(self).items():
        results.append('{0}: {1}'.format(k,v))
    return results

def debug_info(cls):
    cls.debug = info
    return cls

@debug_info
class Person:
    def __init__(self, nombre, birth_year):
        self.name = nombre
        self.birth_year = birth_year
    def say_hi():
        return 'hello there!'

p = Person('john', 1992)
p.debug()

['time: 2021-04-12 00:55:44.812576+00:00',
 'class: Person',
 'id: 0x210eefce708',
 'name: john',
 'birth_year: 1992']

## Dispatching
operator overloading is not possible on python, instead it has Single Dispatching Genric Functions 